This is a playground notebook for database functionality. I migrated this from the higher-level dir so the imports need to be fixed

In [1]:
from app import app, db
from models import User, Venue, Artist
import sqlalchemy as sa

In [2]:
app.app_context().push()

In [6]:
# sqlalchemy only checks for primary key uniqueness
a = Artist(username="rich brian", 
           insta="none", 
           spotify="none", 
           twitter="none",
           facebook="none",
           booking="none",
           location="jakarta", 
           description="rich asf") # maybe it's a mistake to require unique descriptions...
db.session.add(a)
db.session.commit()

In [7]:
# sqlalchemy only checks for primary key uniqueness, but enforcing the uniqueness constraint 
# also makes it impossible to re-add the same artist name, should probably error-handle
a_1 = Artist(username="niki", 
           insta="none", 
           spotify="none", 
           twitter="none",
           facebook="none",
           booking="none",
           location="jakarta", 
           description="i am an incredibly unique artist")

db.session.add(a_1) # so the obj already exists re-adding triggers a rollback error
db.session.commit()

In [8]:
query = sa.select(Artist)
artists = db.session.scalars(query).all() # .all() converst results object into a plain list
print(artists)

for artist in artists: 
    print(artist.id, artist.description)

selected_artist = db.session.get(Artist, 1)
print(selected_artist.description)

[<Artist rich brian>, <Artist niki>]
1 rich asf
2 i am an incredibly unique artist
rich asf


In [9]:
# let's add a venue
v = Venue(location="HITC!", link="no link", date="no date", time="no time")
db.session.add(v)
db.session.commit() #timestamp if it existed would just have a default, the so.relationship implicitly assigns the right Artist by passing in the artist object I created above, sqlalchmey is great at abstracting this logic out

In [10]:
print(v.artists)

[]


In [11]:
try: 
    v.artists = [a, a_1]
    db.session.commit()
except sa.exc.SQLAlchemyError as e:
    print(e)
    print('you should probably rollback your changes')

In [12]:
print(db.session.query(Venue).where(Venue.id == 1).one().artists)

[<Artist rich brian>, <Artist niki>]


In [13]:
print(db.session.query(Artist).where(Artist.id == 1).one().venues)

[<Venue HITC!>]


In [14]:
print(db.session.query(Artist).where(Artist.id == 2).one().venues)

[<Venue HITC!>]


In [16]:
# get all the users in reverse alphabetical order
query = sa.select(Artist).order_by(Artist.username.desc())
print(db.session.scalars(query).all())

[<Artist rich brian>, <Artist niki>]
